In [3]:
#Just some imports you will need
import mlflow
import jwt
import json
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
import os
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [4]:
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])

MLFLOW_TRACKING_URI: http://127.0.0.1:8765


In [5]:
client = mlflow.tracking.MlflowClient()

In [7]:
myprefix=''

from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")

experiment_name = 'demo'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
model_name = 'demo'+'-' + os.environ['DOMINO_PROJECT_NAME']
if myprefix!='':
    experiment_name = myprefix + '-' + experiment_name
print(experiment_name)

DEMO-udaysk-domino_mlops


In [8]:
experiment = client.get_experiment_by_name(name=experiment_name)
if(experiment is None):
    print('Creating experiment ')
    client.create_experiment(name=experiment_name)
    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

Creating experiment 
DEMO-udaysk-domino_mlops


<Experiment: artifact_location='mlflow-artifacts:/mlflow', creation_time=1714028115620, experiment_id='760', last_update_time=1714028115620, lifecycle_stage='active', name='DEMO-udaysk-domino_mlops', tags={'mlflow.domino.dataset_info': '6626601005b050727a03be07-6626601005b050727a03be06',
 'mlflow.domino.environment_id': '61e7b4d727227a585bb9e9e9',
 'mlflow.domino.environment_revision_id': '63c201ceab92a65b16f16c8e',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project_id': '6626600e05b050727a03be02',
 'mlflow.domino.project_name': 'domino_mlops',
 'mlflow.domino.provenance_checkpoint_id': '6628d7f080c5dd6d5f094ed3',
 'mlflow.domino.run_id': '6628d7d505b050727a03bf13',
 'mlflow.domino.run_number': '1',
 'mlflow.domino.user': 'udaysk',
 'mlflow.domino.user_id': '65bd2cd71266902edb95c650',
 'mlflow.source.git.branch': 'main',
 'mlflow.source.git.commit': '68a1993c4af60a7f2d430076acb2167f97ae9c76',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'udaysk'}>

In [9]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [10]:
# Read the wine-quality csv file from the URL
csv_url = (
     "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.7
l1_ratio = 0.6
while(alpha<1):
    with mlflow.start_run():
        print('Start Run')
        print('Alpha : ' + str(alpha))
        print('L1_Ratio : ' + str(l1_ratio))
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(mlflow.get_tracking_uri())
        print(tracking_url_type_store)
        # Model registry does not work with file store
        if tracking_url_type_store != "file":
            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #mlflow.sklearn.log_model(lr, os.environ['DOMINO_STARTING_USERNAME']+"/model", registered_model_name="DEMO-ElasticnetWineModel-11")
            mlflow.sklearn.log_model(lr, "gsk_demo_model", registered_model_name=model_name)

        else:        
            mlflow.sklearn.log_model(lr,  model_name)
        print(mlflow.get_artifact_uri())
        alpha=alpha+0.1
        l1_ratio = l1_ratio + 0.05        
        

Start Run
Alpha : 0.7
L1_Ratio : 0.6
Elasticnet model (alpha=0.700000, l1_ratio=0.600000):
  RMSE: 0.7555128766818596
  MAE: 0.6334930689742281
  R2: 0.037345942457275516
http://127.0.0.1:8765
http


Successfully registered model 'DEMO-domino_mlops'.
2024/04/25 06:55:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DEMO-domino_mlops, version 1
Created version '1' of model 'DEMO-domino_mlops'.


mlflow-artifacts:/mlflow/ab700ede434e45bfb1667529a72d66ac/artifacts
Start Run
Alpha : 0.7999999999999999
L1_Ratio : 0.65
Elasticnet model (alpha=0.800000, l1_ratio=0.650000):
  RMSE: 0.7567822485433978
  MAE: 0.6355053405206407
  R2: 0.034108426456301966
http://127.0.0.1:8765
http


Registered model 'DEMO-domino_mlops' already exists. Creating a new version of this model...
2024/04/25 06:55:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DEMO-domino_mlops, version 2
Created version '2' of model 'DEMO-domino_mlops'.


mlflow-artifacts:/mlflow/6dfe62dd7be04d708c0ce18e67ed883a/artifacts
Start Run
Alpha : 0.8999999999999999
L1_Ratio : 0.7000000000000001
Elasticnet model (alpha=0.900000, l1_ratio=0.700000):
  RMSE: 0.7570080643851105
  MAE: 0.6362102528360067
  R2: 0.0335319167389968
http://127.0.0.1:8765
http


Registered model 'DEMO-domino_mlops' already exists. Creating a new version of this model...
2024/04/25 06:55:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DEMO-domino_mlops, version 3
Created version '3' of model 'DEMO-domino_mlops'.


mlflow-artifacts:/mlflow/301832768b014b8a875721b9bd65848c/artifacts
Start Run
Alpha : 0.9999999999999999
L1_Ratio : 0.7500000000000001
Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.757266418646812
  MAE: 0.6369790527846166
  R2: 0.032872125205521296
http://127.0.0.1:8765
http


Registered model 'DEMO-domino_mlops' already exists. Creating a new version of this model...
2024/04/25 06:55:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DEMO-domino_mlops, version 4
Created version '4' of model 'DEMO-domino_mlops'.


mlflow-artifacts:/mlflow/ad8ce4c900804d74baf24bb6a14880b8/artifacts


In [14]:
!pip show mlflow

Name: mlflow
Version: 2.12.1
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: 
Author: 
Author-email: 
License: Copyright 2018 Databricks, Inc.  All rights reserved.
        
        				Apache License
                                   Version 2.0, January 2004
                                http://www.apache.org/licenses/
        
           TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION
        
           1. Definitions.
        
              "License" shall mean the terms and conditions for use, reproduction,
              and distribution as defined by Sections 1 through 9 of this document.
        
              "Licensor" shall mean the copyright owner or entity authorized by
              the copyright owner that is granting the License.
        
              "Legal Entity" shall mean the union of the acting entity and all
              other entities that control, are controlled by, or are under common
        